<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/MEDICAL_BoltzmannMachine_DEMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ENVIROMENT

In [ ]:
!pip install --upgrade ipykernel -q
!pip install --upgrade IPython -q
!pip install datasets -q
!pip install gym -q

!pip install torchviz -q
!pip install torch-tb-profiler -q
!pip install tensorboard -q
!pip install datasets -q

!pip install --upgrade hyperopt -q
!pip install --upgrade setuptools -q

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning, module="ipykernel.ipkernel")
warnings.filterwarnings("ignore", category=DeprecationWarning, module="pkg_resources")
warnings.filterwarnings("ignore", category=DeprecationWarning, module="torch.nn.functional")
warnings.filterwarnings("ignore", category=DeprecationWarning, module="hyperopt")
# full and complete code corrected and no more placeholders

## GPU SETUP

In [ ]:
!nvidia-smi

Wed Feb 12 04:32:18 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      Off |   00000000:00:03.0 Off |                    0 |
| N/A   48C    P0             29W /   72W |     251MiB /  23034MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
import torch
print(f"Number of GPUs: {torch.cuda.device_count()}")

Number of GPUs: 1


In [ ]:
import torch

# Check for GPU and set device
if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f"Using GPU: {torch.cuda.get_device_name(device)}")
else:
    device = torch.device('cpu')
    print("Using CPU")

# Create two tensors on the device
a = torch.randn(1000, 1000, device=device)
b = torch.randn(1000, 1000, device=device)

# Perform a matrix multiplication on the device
c = torch.matmul(a, b)

# Print the result (to force computation)
print(c.sum())

Using GPU: NVIDIA L4
tensor(-14796.9902, device='cuda:0')


In [ ]:
import torch
print(torch.cuda.is_available())

True


## MEDICAL

In [ ]:
!pip install --upgrade ipykernel -q
!pip install --upgrade IPython -q
!pip install datasets -q
!pip install gym -q
!pip install torchviz -q
!pip install torch-tb-profiler -q
!pip install tensorboard -q
!pip install datasets -q
!pip install --upgrade hyperopt -q
!pip install --upgrade setuptools -q

DATA LOAD

In [ ]:
!pip install kagglehub --upgrade -q

https://www.kaggle.com/datasets/andrewmvd/heart-failure-clinical-data?resource=download

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("andrewmvd/heart-failure-clinical-data")

print("Path to dataset files:", path)

import pandas as pd

# Assuming 'heart_failure_clinical_records_dataset.csv' is the file name in the downloaded directory:
data = pd.read_csv(path + "/heart_failure_clinical_records_dataset.csv")

100%|██████████| 3.97k/3.97k [00:00<00:00, 4.23MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/andrewmvd/heart-failure-clinical-data/versions/1


In [ ]:
data

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,62.0,0,61,1,38,1,155000.00,1.1,143,1,1,270,0
295,55.0,0,1820,0,38,0,270000.00,1.2,139,0,0,271,0
296,45.0,0,2060,1,60,0,742000.00,0.8,138,0,0,278,0
297,45.0,0,2413,0,38,0,140000.00,1.4,140,1,1,280,0


In [ ]:
import gym
import torch
import torch.nn as nn
import numpy as np
from tqdm import tqdm
import pandas as pd
import kagglehub
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
import warnings
import sys
import logging

# Suppress warnings
warnings.filterwarnings("ignore", category=DeprecationWarning, module="pkg_resources")
warnings.filterwarnings("ignore", category=DeprecationWarning, module="hyperopt")
logging.captureWarnings(True)
logger = logging.getLogger()
logger.setLevel(logging.ERROR)

# Check for GPU and set device
if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f"Using GPU: {torch.cuda.get_device_name(device)}")
else:
    device = torch.device('cpu')
    print("Using CPU")

# Boltzmann Machine Class
class BoltzmannMachine(nn.Module):
    def __init__(self, num_visible, num_hidden):
        super(BoltzmannMachine, self).__init__()
        self.num_visible = num_visible
        self.num_hidden = num_hidden
        self.weights = nn.Parameter(torch.randn(num_visible, num_hidden) * 0.1) # Define weights as an instance attribute
        self.hidden_bias = nn.Parameter(torch.zeros(num_hidden)) # Define hidden_bias as an instance attribute
        self.visible_bias = nn.Parameter(torch.zeros(num_visible)) # Define visible_bias as an instance attribute
        self.to(device) # Move to device after defining parameters

    # ... (rest of the class methods remain the same)
    def sample_hidden(self, visible):
        hidden_activations = torch.matmul(visible, self.weights) + self.hidden_bias
        hidden_probs = torch.sigmoid(hidden_activations)
        hidden_states = torch.bernoulli(hidden_probs)
        return hidden_states

    def sample_visible(self, hidden):
        visible_activations = torch.matmul(hidden, self.weights.t()) + self.visible_bias
        visible_probs = torch.sigmoid(visible_activations)
        visible_states = torch.bernoulli(visible_probs)
        return visible_states

    def train(self, data, epochs=10, learning_rate=0.1):
        data = torch.tensor(data, dtype=torch.float32, device=self.weights.device) # Move data to the same device as weights
        for epoch in range(epochs):
            pos_hidden_probs = torch.sigmoid(torch.matmul(data, self.weights) + self.hidden_bias)
            pos_hidden_states = torch.bernoulli(pos_hidden_probs)
            pos_associations = torch.matmul(data.t(), pos_hidden_probs)

            neg_visible_states = self.sample_visible(pos_hidden_states)
            neg_hidden_probs = torch.sigmoid(torch.matmul(neg_visible_states, self.weights) + self.hidden_bias)
            neg_associations = torch.matmul(neg_visible_states.t(), neg_hidden_probs)

            # Update weights, hidden_bias, and visible_bias using in-place operations
            self.weights.data += learning_rate * (pos_associations - neg_associations) / data.shape[0]
            self.hidden_bias.data += learning_rate * (torch.sum(pos_hidden_probs, dim=0) - torch.sum(neg_hidden_probs, dim=0)) / data.shape[0]
            self.visible_bias.data += learning_rate * (torch.sum(data, dim=0) - torch.sum(neg_visible_states, dim=0)) / data.shape[0]


# Medical Environment Class
class MedicalEnv(gym.Env):
    def __init__(self, data, num_features=12):  # num_features should match your dataset
        self.data = data
        self.action_space = gym.spaces.Discrete(2)  # 0: No Death, 1: Death
        self.observation_space = gym.spaces.Dict({
            'features': gym.spaces.Box(low=self.data.min().values[:-1],
                                      high=self.data.max().values[:-1],
                                      shape=(num_features,),
                                      dtype=np.float32),
        })
        self.current_patient_index = 0
        self.reset()

    def reset(self):
        self.current_patient_index = (self.current_patient_index + 1) % len(self.data)
        self.features = self.data.iloc[self.current_patient_index].drop("DEATH_EVENT").values.astype(np.float32)
        return self._get_obs()

    def step(self, action):
        actual_outcome = self.data.iloc[self.current_patient_index]["DEATH_EVENT"]
        reward = self.reward_function(action, actual_outcome)
        done = True  # Episode ends after each step
        return self._get_obs(), reward, done, {}

    def reward_function(self, action, actual_outcome):
        if action == actual_outcome:
            return 1  # Reward for correct prediction
        else:
            return -1  # Penalty for incorrect prediction

    def _get_obs(self):
        return {
            'features': self.features,
        }

# Q-Learning Agent Class
class QLearningAgent:
    def __init__(self, observation_space, action_space, learning_rate=0.01, discount_factor=0.9, epsilon=0.2,
                 epsilon_decay=0.995, device=None):
        self.observation_space = observation_space
        self.action_space = action_space
        self.learning_rate = learning_rate
        self.discount_factor = discount_factor
        self.epsilon = epsilon
        self.epsilon_decay = epsilon_decay
        self.device = device

        # Q-table (using a dictionary to handle continuous features)
        self.q_table = {}  # Store Q-values for state-action pairs
        self.returns = {}  # Store returns for each state-action pair

    def get_action(self, state):
        state_tuple = tuple(state['features'].flatten())  # Convert state to a tuple

        if state_tuple not in self.q_table:
            self.q_table[state_tuple] = np.zeros(self.action_space.n, dtype=np.float32)
            self.q_table[state_tuple][:] = 0.1  # Initialize with small positive values

        if np.random.rand() < self.epsilon:
            return self.action_space.sample()
        else:
            q_values = self.q_table[state_tuple]
            action = np.argmax(q_values)
            return action

    def update(self, episode):
        states, actions, rewards = zip(*episode)
        G = 0
        for t in reversed(range(len(states))):
            G = self.discount_factor * G + rewards[t]
            state_tuple = tuple(states[t]['features'].flatten())
            state_action = (state_tuple, actions[t])

            if state_action not in self.returns:
                self.returns[state_action] = []

            self.returns[state_action].append(G)
            self.q_table[state_tuple][actions[t]] = np.mean(self.returns[state_action])

        self.epsilon = max(0.01, self.epsilon * self.epsilon_decay)

# Download the dataset
path = kagglehub.dataset_download("andrewmvd/heart-failure-clinical-data")
print("Path to dataset files:", path)

# Load the dataset
data = pd.read_csv(path + "/heart_failure_clinical_records_dataset.csv")

# Create environment and Boltzmann machine
env = MedicalEnv(data)
bm = BoltzmannMachine(num_visible=len(data.columns) - 1, num_hidden=50).to(device)

# Generate random training data for the Boltzmann machine
training_data = data.drop(columns=["DEATH_EVENT"]).values.astype(np.float32)
training_data = torch.tensor(training_data, dtype=torch.float32, device=device)

# Define the objective function for hyperopt
def objective(params):
    agent = QLearningAgent(env.observation_space, env.action_space,
                           learning_rate=params['learning_rate'],
                           discount_factor=params['discount_factor'],
                           epsilon=params['epsilon'],
                           epsilon_decay=params['epsilon_decay'],
                           device=device)

    for episode in range(500):
        state = env.reset()
        done = False
        episode_data = []
        while not done:
            # Get Boltzmann machine input
            bm_input = torch.tensor(state['features'], dtype=torch.float32, device=device)
            # Sample hidden state
            hidden_state = bm.sample_hidden(bm_input)

            # Combine features and hidden state for Q-learning state
            processed_state = {
                'features': state['features'],
                'hidden_state': hidden_state.cpu().detach().numpy().flatten()  # Add hidden state to state
            }

            # Ensure processed_state has the expected format for get_action
            processed_state_for_action = {'features': np.concatenate([processed_state['features'], processed_state['hidden_state']])}
            action = agent.get_action(processed_state_for_action)  # Pass the combined state to get_action

            next_state, reward, done, _ = env.step(action)
            episode_data.append((processed_state_for_action, action, reward))  # Store the combined state
            state = next_state
        agent.update(episode_data)

    # Evaluation for hyperopt
    total_eval_reward = 0
    for _ in range(100):  # Evaluation episodes
        state = env.reset()
        done = False
        while not done:
            # Get Boltzmann machine input
            bm_input = torch.tensor(state['features'], dtype=torch.float32, device=device)
            # Sample hidden state
            hidden_state = bm.sample_hidden(bm_input)

            # Combine features and hidden state for Q-learning state
            processed_state = {
                'features': state['features'],
                'hidden_state': hidden_state.cpu().detach().numpy().flatten()  # Add hidden state to state
            }

            # Ensure processed_state has the expected format for get_action
            processed_state_for_action = {'features': np.concatenate([processed_state['features'], processed_state['hidden_state']])}

            action = agent.get_action(processed_state_for_action)  # Pass the combined state to get_action
            next_state, reward, done, _ = env.step(action)
            total_eval_reward += reward
            state = next_state

    avg_eval_reward = total_eval_reward / 100
    return {'loss': -avg_eval_reward, 'status': STATUS_OK}  # hyperopt minimizes, so negate reward

# Define the hyperparameter search space
space = {
    'discount_factor': hp.uniform('discount_factor', 0.7, 0.99),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.001), np.log(0.1)),
    'epsilon': hp.uniform('epsilon', 0.1, 0.3),
    'epsilon_decay': hp.uniform('epsilon_decay', 0.99, 0.999)
}

# Perform hyperparameter optimization
trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=50,  # Number of hyperparameter evaluations
            trials=trials)

print(f"Best hyperparameters: {best}")

# Create the best agent with the found hyperparameters
best_agent = QLearningAgent(env.observation_space, env.action_space,
                           discount_factor=best['discount_factor'],
                           learning_rate=best['learning_rate'],
                           epsilon=best['epsilon'],
                           epsilon_decay=best['epsilon_decay'],
                           device=device)

# Final Training with Best Hyperparameters
for episode in tqdm(range(2000), desc="Final Training Progress"):
    state = env.reset()
    done = False
    episode_data = []
    while not done:
        # Get Boltzmann machine input
        bm_input = torch.tensor(state['features'], dtype=torch.float32, device=device)
        # Sample hidden state
        hidden_state = bm.sample_hidden(bm_input)

        # Combine features and hidden state for Q-learning state
        processed_state = {
            'features': state['features'],
            'hidden_state': hidden_state.cpu().detach().numpy().flatten()  # Add hidden state to state
        }

        # Ensure processed_state has the expected format for get_action
        processed_state_for_action = {'features': np.concatenate([processed_state['features'], processed_state['hidden_state']])}
        action = best_agent.get_action(processed_state_for_action)  # Pass the combined state to get_action

        next_state, reward, done, _ = env.step(action)
        episode_data.append((processed_state_for_action, action, reward))  # Store the combined state
        state = next_state
    best_agent.update(episode_data)
    if episode % 50 == 0:
        # Sample a batch from training_data
        batch_indices = torch.randint(0, len(training_data), (64,))  # Batch size of 64
        batch_data = training_data[batch_indices]

        bm.train(batch_data, epochs=10, learning_rate=0.1)  # Train with the batch

        #bm.train(training_data, epochs=10, learning_rate=0.1)

# Evaluation Phase
total_eval_reward = 0
correct_predictions = 0
for episode in tqdm(range(100), desc="Evaluation Progress"):
    state = env.reset()
    done = False
    while not done:
        # Get Boltzmann machine input
        bm_input = torch.tensor(state['features'], dtype=torch.float32, device=device)
        # Sample hidden state
        hidden_state = bm.sample_hidden(bm_input)

        # Combine features and hidden state for Q-learning state
        processed_state = {
            'features': state['features'],
            'hidden_state': hidden_state.cpu().detach().numpy().flatten()  # Add hidden state to state
        }

        # Ensure processed_state has the expected format for get_action
        processed_state_for_action = {'features': np.concatenate([processed_state['features'], processed_state['hidden_state']])}
        action = best_agent.get_action(processed_state_for_action)  # Pass the combined state to get_action

        next_state, reward, done, _ = env.step(action)
        total_eval_reward += reward

        # Compare action (prediction) with actual outcome
        # Use env.data instead of self.data to access the data attribute of the env object
        actual_outcome = env.data.iloc[env.current_patient_index]["DEATH_EVENT"]  # Access actual outcome from data
        if action == actual_outcome:
            correct_predictions += 1
        state = next_state

avg_eval_reward = total_eval_reward / 100
accuracy = correct_predictions / 100
print(f"\nAverage evaluation reward: {avg_eval_reward}")
print(f"Accuracy: {accuracy}")

Using GPU: NVIDIA L4
Path to dataset files: /root/.cache/kagglehub/datasets/andrewmvd/heart-failure-clinical-data/versions/1
100%|██████████| 50/50 [00:22<00:00,  2.25trial/s, best loss: -1.0]
Best hyperparameters: {'discount_factor': 0.8429520934337369, 'epsilon': 0.12076181550519467, 'epsilon_decay': 0.9901707566842393, 'learning_rate': 0.00638990985650379}


Evaluation Progress: 100%|██████████| 100/100 [00:00<00:00, 1338.60it/s]


Average evaluation reward: 0.98
Accuracy: 0.99
